# Technical Assignment - IKEA DA

Prepared by **Michael Zhang** | 2021 Mar | @AMSTERDAM

## 0. Executive Summary

### Question 1: Propose a segmentation analysis

Seg A, B, C

### Question 3: Key segment(s) to focus to increase sales

Seg A
based on A, B, C reasons
* Increase sales is more of a short-term goal -> with the info I have, either focus on the large seg with potential to upsell (small improvement but big moneytory potential), or niche seg with great %growth potential

### Question 2: Potential data points to improve the analysis

* Unit info: understand if the product is of high unit price (which may not be purchased again in a short term)
* Category info: similar to above; also make it difficult to interpret the clustering
* Transactional info: what does it buy together per order -> basket analysis, frequency analysis, recency (3D of CLV or CV)


### Question 4: Prediction of the value in "HaveChildren" with vitualization

## 1. Data Import and Exploration

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from kneed import KneeLocator
from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler
# from sklearn-extras
# from sk-learn-extra

In [ ]:
# df = pd.read_csv('RetailCustomerSales.csv',sep = ';', dtype = {'CustomerID':'category',
#                                                                'ItemID':'category',
#                                                                'Sex':'category',
#                                                                'Age':'category',
#                                                                'Profession':'category',
#                                                                'CityType':'category',
#                                                                'YearsInCity':'category',
#                                                                'HaveChildren':'category',
#                                                                'ItemCategory1':'category',
#                                                                'ItemCategory2':'category',
#                                                                'ItemCategory3':'category'})

df = pd.read_csv('RetailCustomerSales.csv',sep = ';', dtype = {'CustomerID':'str',
                                                               'ItemID':'str',
                                                               'Sex':'str',
                                                               'Age':'str',
                                                               'Profession':'str',
                                                               'CityType':'str',
                                                               'YearsInCity':'str',
                                                               'HaveChildren':'str',
                                                               'ItemCategory1':'str',
                                                               'ItemCategory2':'str',
                                                               'ItemCategory3':'str'})

In [ ]:
display(df)

In [ ]:
df.info()

In [ ]:
# Check Null value
print(df.isnull().sum())

In [ ]:
df.describe()

In [ ]:
# len(df['CustomerID'].unique())
consumer_item = df.groupby('CustomerID')['ItemID'].nunique().sort_values()
print(consumer_item)

In [ ]:
# len(df['CustomerID'].unique())
consumer_spend = df.groupby('CustomerID')['Amount'].sum().sort_values()
print(consumer_spend)

#plot: check outliers

In [ ]:
#plot: check potential outliers and distribution
plt.style.use("fivethirtyeight")
plt.hist(consumer_spend, bins = 1000)
plt.xlabel("Spending")
plt.ylabel("ConsumerID Count")
plt.show()

In [ ]:
consumer_spend.describe()

**Based on above chart, turn spending per consumer into log scale skew to positive side (check the ref)**

In [ ]:
# len(df['ItemID'].unique())
item_consumer = df.groupby('ItemID')['CustomerID'].nunique().sort_values()
print(item_consumer)

In [ ]:
#plot popular product and check distribution
plt.style.use("fivethirtyeight")
plt.hist(item_consumer, bins = 1000)
plt.xlabel("Buyer count")
plt.ylabel("Item Count")
plt.show()

In [ ]:
item_consumer.describe()

In [ ]:
# len(df['ItemID'].unique())
item_sales = df.groupby('ItemID')['Amount'].sum().sort_values()
print(item_sales)

#plot high sales value

In [ ]:
#plot high total sales value and check distribution
plt.style.use("fivethirtyeight")
plt.hist(item_sales, bins = 1000)
plt.xlabel("Total Sales")
plt.ylabel("Item Count")
plt.show()

In [ ]:
item_sales.describe()

In [ ]:
# df['Sex'].unique()
sex = df.groupby('Sex')['CustomerID'].nunique().sort_index()
print(sex)

In [ ]:
# pd.Series(list(df['Age'].unique().astype('str'))).sort_values() # as category
# df['Sex'].unique()
age = df.groupby('Age')['CustomerID'].nunique().sort_index()
print(age)

In [ ]:
# df['CityType'].unique()
CityType = df.groupby('CityType')['CustomerID'].nunique().sort_index()
print(CityType)

In [ ]:
# df['YearsInCity'].unique() # as category
# YearsInCity = df['YearsInCity'].unique()
# print(np.sort(YearsInCity))
YearsInCity = df.groupby('YearsInCity')['CustomerID'].nunique().sort_index()
print(YearsInCity)

In [ ]:
# pd.Series(list(df['Profession'].unique().astype('int'))).sort_values() # as category
# Profession = df['Profession'].nunique()
# print(np.sort(Profession))
Profession = df.groupby('Profession')['CustomerID'].nunique().sort_index()
print(Profession)

In [ ]:
#plt to see if there is significant inbalanced value
plt.style.use("fivethirtyeight")
plt.bar(Profession.index,Profession)
plt.xlabel("Profession")
plt.ylabel("Consumer Count")
plt.show()

**Based on above chart, one-hot coding may skew heavily to the rare professions sucha as 8, 18, and 19. May consider use k-prototype/k-mode to avoid such issue**

In [ ]:
# df['HaveChildren'].unique()
# df.loc[df['HaveChildren'].isnull(),'CustomerID'].nunique()
HaveChildren = df.groupby('HaveChildren',dropna=False)['CustomerID'].nunique().sort_index()
print(HaveChildren)

**212 consumers are without Children info. Will need to run clustering twice - with and without NaN**

In [ ]:
# df['ItemCategory1'].unique()
ItemCategory1 = df.groupby('ItemCategory1',dropna=False)['ItemID'].nunique().sort_values()
print(ItemCategory1)

In [ ]:
# df['ItemCategory2'].unique()
ItemCategory2 = df.groupby('ItemCategory2',dropna=False)['ItemID'].nunique().sort_values()
print(ItemCategory2)

In [ ]:
# df['ItemCategory3'].unique()
ItemCategory3 = df.groupby('ItemCategory3',dropna=False)['ItemID'].nunique().sort_values()
print(ItemCategory3)

In [ ]:
df.loc[df['ItemCategory1'] == df['ItemCategory2'],'ItemID'].unique()

In [ ]:
ItemCombo = df.groupby(['ItemCategory1','ItemCategory2','ItemCategory3'],
                       dropna=False)['ItemID'].nunique().sort_values()
print(ItemCombo)

In [ ]:
#plt to see how inbalanced it will be if converted to one hot encoded
ItemCombo_reindex = ItemCombo.reset_index()
plt.style.use("fivethirtyeight")
plt.bar(ItemCombo_reindex.index,ItemCombo_reindex.ItemID)
plt.xlabel("Category Combo")
plt.ylabel("Item Count")
plt.show()

In [ ]:
ItemComboSales = df.groupby(['ItemCategory1','ItemCategory2','ItemCategory3'],
                       dropna=False)['Amount'].sum().sort_values()
print(ItemComboSales)

In [ ]:
#plt to see how inbalanced it will be if converted to one hot encoded
ItemComboSales_reindex = ItemComboSales.sort_index().reset_index()
ItemComboSales_reindex['CatCombo'] = ItemComboSales_reindex.index
plt.style.use("fivethirtyeight")
plt.bar(ItemComboSales_reindex.CatCombo,ItemComboSales_reindex.Amount)
plt.xlabel("Category Combo")
plt.ylabel("Sales")
plt.show()

In [ ]:
print(ItemComboSales_reindex.head())

**Either convert to 18 categories with overlaps (double count if one item falls to more than one category or use the 235 category combinations; run them with and without such. With 235 combo, also need to be logged**

## 2. Segmentation

Some options I have/decisions I have to make:
1. If there is an outlier -> looks like the data is quite clean
2. To standardize or not (for cat amount and the total amount)  
using mapping and replace  
CustomerID -> group by  
ItemID -> drop  
Sex -> to -1 and 1; group by  
Age -> take the medium and rescale (0-17 as 16; 55+ as 60)/ rescale  
Profession -> one-hot encoding?  
CityType -> 1/A, 0/B, -1/C? 
YearsInCity -> convert to numbers (4+ as 4); group by  
(hold on) HaveChildren -> to -1(N), 0 (NULL) and 1(Y); group by   
[optional] 18 Category  
235 cateogires -> both take log and rescale  
Amount -> tke the log and rescale  
3. 18 main cat (with amount overlap and also corelation between cats) or 200+ combo cat
4. can I use k-means++ to set the centroid
5. k-means, k-prototype, k-mode/medroid
6. what evaluation metrics to use: EC, explanationary, SW

### 2.1 Data Preparation

In [ ]:
df_demo = df.groupby(['CustomerID','Sex','Age','Profession',
                      'CityType','YearsInCity','HaveChildren'])['Amount']
                      .sum().rename(columns={'Amount':'TotalSpending'})

In [ ]:
## Create mapping for sex, age, CityType, YearsInCity
replace_map = {'sex': {'M': 1, 'F': 0},
               'age': {'0-17': 16, '18-25': 21.5, '26-35': 30.5, '36-45': 40.5,
                       '46-50': 48, '51-55': 53, '55+': 55},
               'CityType': {'A': 1, 'B': 2, 'C': 3},
               'YearsInCity': {'4+':4}}

In [ ]:
## Replace the value with the mapping
df_replaced = df_demo.copy().drop('Profession')
df_replaced.replace(replace_map, inplace=True)

print(df_replaced.head())

In [ ]:
df_replaced.info()

In [ ]:
## Create one-hot encoding for professions

df_prof = df_demo.copy().drop(['Sex','Age','CityType','YearsInCity','HaveChildren'])
df_prof = pd.get_dummies(df_prof, columns=['Profession'], prefix = ['Prof'])

print(df_prof.head())

In [ ]:
## Create one-hot encoding for cat combo
# df_cat = df.groupby(['CustomerID','ItemCategory1',
#                      'ItemCategory2','ItemCategory3'])['Amount'].sum()

df_combo = df.copy()

df_combo['CatCombo'] = df_combo[['ItemCategory1','ItemCategory2','ItemCategory3']]
                       .agg('_'.join, axis=1)

df_combo = pd.pivot_table(df_combo, values='Amount', columns=['CatCombo']
                          , aggfunc=np.sum, fill_value=0)

print(df_combo.head())

In [ ]:
## Complie all the features together
df_compile = df_replaced.join([df_prof, df_combo], on=['CustomerID'])

In [ ]:
print(df_compile.head(20))

In [ ]:
df_compile.info()

In [ ]:
df_compile.describe()

In [ ]:
## K-means
## Ref: https://stats.stackexchange.com/questions/133656/how-to-understand-the-drawbacks-of-k-means
## Ref: https://realpython.com/k-means-clustering-python/

## PAM (Partitioning Around Medoids)
## Ref: https://towardsdatascience.com/clustering-datasets-having-both-numerical-and-categorical-variables-ed91cdca0677

## Categorical data
## Ref: https://www.datacamp.com/community/tutorials/categorical-data
## Ref: https://pbpython.com/categorical-encoding.html  
## Ref: https://towardsdatascience.com/all-about-categorical-variable-encoding-305f3361fd02

In [ ]:
features, true_labels = make_blobs(
    n_samples=200,
    centers=3,
    cluster_std=2.75,
    random_state=42
)

In [ ]:
features[:5]

In [ ]:
true_labels[:5]

#### Scale the features

In [ ]:
scaler = StandardScaler()
scaled_features = scaler.fit_transform(features)

In [ ]:
scaled_features[:5]

#### Set the parameter of K-means clustering

In [1]:
kmeans = KMeans(
    init="random",
    n_clusters=3,
    n_init=10,
    max_iter=300,
    random_state=42
)

NameError: name 'KMeans' is not defined

In [ ]:
kmeans.fit(scaled_features)

#### Result

In [ ]:
# The lowest SSE value
kmeans.inertia_

In [ ]:
# Final locations of the centroid
kmeans.cluster_centers_

In [ ]:
# The number of iterations required to converge
kmeans.n_iter_

In [ ]:
kmeans.labels_[:5]

#### Evaluation - SSE

In [ ]:
kmeans_kwargs = {
    "init": "random",
    "n_init": 10,
    "max_iter": 300,
    "random_state": 42,
}

# A list holds the SSE values for each k
sse = []
for k in range(1, 11):
    kmeans = KMeans(n_clusters=k, **kmeans_kwargs)
    kmeans.fit(scaled_features)
    sse.append(kmeans.inertia_)

In [ ]:
## Determine the elbow from plot
plt.style.use("fivethirtyeight")
plt.plot(range(1, 11), sse)
plt.xticks(range(1, 11))
plt.xlabel("Number of Clusters")
plt.ylabel("SSE")
plt.show()

In [ ]:
## Determine the elbow programmatically
kl = KneeLocator(
    range(1, 11), sse, curve="convex", direction="decreasing"
)

kl.elbow

#### Evaluation - silhouette coefficients

In [ ]:
# A list holds the silhouette coefficients for each k
silhouette_coefficients = []

# Notice you start at 2 clusters for silhouette coefficient
for k in range(2, 11):
    kmeans = KMeans(n_clusters=k, **kmeans_kwargs)
    kmeans.fit(scaled_features)
    score = silhouette_score(scaled_features, kmeans.labels_)
    silhouette_coefficients.append(score)

In [ ]:
## Determine the max(sc) from plot
plt.style.use("fivethirtyeight")
plt.plot(range(2, 11), silhouette_coefficients)
plt.xticks(range(2, 11))
plt.xlabel("Number of Clusters")
plt.ylabel("Silhouette Coefficient")
plt.show()